In [1]:
import os 
import tiktoken
import openai 
from dotenv import load_dotenv
from langchain_openai import AzureChatOpenAI


In [2]:
env_path = ".env"
load_dotenv(dotenv_path=env_path,override=True)
api_type = "azure"
endpoint = os.environ.get("AZURE_OPENAI_ENDPOINT")
api_key = os.environ.get("AZURE_OPENAI_API_KEY")
api_version = '2023-07-01-preview'
deployment_name = os.environ.get("AZURE_DEPLOYMENT_NAME")

encoding = tiktoken.get_encoding('cl100k_base')

set the temperature


In [3]:
azure_llm = AzureChatOpenAI(
            deployment_name=deployment_name,
            temperature=0,
            max_tokens=None,
            api_version="2023-07-01-preview",
            azure_endpoint =endpoint,
            api_key=api_key,

        )
promt = """What would be a programmers favorite Food? give me the single food name only."""
messages =[{"role":"user","content":promt}]

for x in range(10):
    response= azure_llm.invoke(messages)
    print(response.content)

Pizza
Pizza
Pizza
Pizza
Pizza
Pizza
Pizza
Pizza
Pizza
Pizza


initialize model

In [4]:
import os 
import tiktoken
import openai 
from dotenv import load_dotenv
from langchain_openai import AzureChatOpenAI

env_path = ".env"
load_dotenv(dotenv_path=env_path,override=True)
api_type = "azure"
endpoint = os.environ.get("AZURE_OPENAI_ENDPOINT")
api_key = os.environ.get("AZURE_OPENAI_API_KEY")
api_version = '2023-07-01-preview'
deployment_name = os.environ.get("AZURE_DEPLOYMENT_NAME")

encoding = tiktoken.get_encoding('cl100k_base')

azure_llm = AzureChatOpenAI(
            deployment_name=deployment_name,
            temperature=0,
            max_tokens=None,
            api_version="2023-07-01-preview",
            azure_endpoint =endpoint,
            api_key=api_key,

        )
def get_completion(prompt, model=azure_llm):
    messages =[{"role":"user","content":prompt}]
    response=model.invoke(messages)
    return response.content

In [23]:
prompt = """What would be a programmers favorite Food? give me the single food name only."""
get_completion(prompt)

'Pizza'

## Rules

### 1.) Be specific

- 3 cizgiden olusan 2 satir arasindaki text i bir satira indirecek sekilde özetle.
- model e ne yapmamasi gerektigini söylemek yerine ne yapmasi gerektigini söylemek daha iyi sonuc verir: bir cümleden fazla yazma demek yerine 1 cümle yaz demek.


In [24]:
sampletext = "Prompt engineering is a process in machine learning, specifically in Natural Language Processing (NLP), where prompts are designed to guide the model to provide the desired output. This technique is particularly used in transformer-based models like GPT-3. The quality of the prompt can significantly affect the model's performance. It involves crafting questions or statements in a way that helps the model understand the context and give relevant responses. It can be considered as a form of 'soft coding' to instruct an AI model."

prompt = f"""
Summarize the text between two lines of triple dashes into a single sentence.
---
{sampletext}
---
"""
response = get_completion(prompt)
print(response)

Prompt engineering in machine learning, specifically in Natural Language Processing, involves designing prompts to guide transformer-based models like GPT-3 to provide the desired output, and the quality of the prompt significantly affects the model's performance.


⚠️Yukarıdaki yöntem Prompt Enjeksiyon u da önler!  Sınırlayıcı stratejinizi bilmedikleri sürece.

## 2.) Self-check

- belirtilen göreve uygun bir input gelmediginde "No Text Provided" gibi farkli bir output vermeli

In [25]:
sampletext = "1+1*2+3? equals what?"

prompt = f"""
Summarize the text between two lines of triple dashes into a single sentence.
---
{sampletext}
---
"""
response = get_completion(prompt)
print(response)

The equation 1+1*2+3? is being asked for its result.


In [26]:
sampletext = "1+1*2+3? equals what?"

prompt = f"""
Summarize the text between two lines of triple dashes into a single sentence. Say "No Text Provided" if there is no text between the triple dashes.
---
{sampletext}
---
"""
response = get_completion(prompt)
print(response)

No Text Provided


## 3.) Few Shot Prompting
References:
- https://arxiv.org/pdf/2302.13971.pdf
- https://arxiv.org/abs/2001.08361
- https://arxiv.org/abs/2005.14165


In [27]:
prompt = f"""
Answer questions in a consistent style

<q>: How can I learn to be a good programmer?
<a>: In the realm of codes, where logic dances and sways,
A quest to be a good programmer, you wish to pave your way.
Fear not, for I shall guide you, in an enchanting lyrical play.

Dive deep into the ocean of languages, where C++, Java, and Python lay,
Choose your vessel wisely, let passion lead your way.

<q>: How do I maximize my credit card rewards?
"""

response = get_completion(prompt)
print(response)

<a>: To maximize your credit card rewards and earn the most,
Use your card for everyday purchases, without any cost.
Look for cards with bonus categories and sign-up bonuses too,
And always pay off your balance on time, that's key to do.


## 4.) Time to think/ Chain-of-Thought (CoT) Prompting
- adim adim cözüme gitmesini sagla

In [30]:
# bu hali ile komplex soru larda yanlis cevap verebilir. Burada asama asama sonuca gitmesi saglanmali
prompt = f"""
1+1+(2*2^2(2+3+3*5))=?
"""
response = get_completion(prompt)
print(response)

1+1+(2*2^2(2+3+3*5))
= 1+1+(2*2^2(2+3+15))
= 1+1+(2*2^2(20))
= 1+1+(2*2*20)
= 1+1+(2*40)
= 1+1+80
= 82


In [31]:
calculation = f"""
1+1+(2*2^2(2+3+3*5))=?
"""
prompt = f"""
Follow the steps below and provide your answer for each step
1. Calculate the section between parantheses
2. replace the paranteses with the result
3. calculate the result

---
{calculation}
---
"""

response = get_completion(prompt)
print(response)

1. Calculate the section between parantheses
2*2^2(2+3+3*5) = 2*2^2(2+3+15) = 2*2^2(20) = 2*4*20 = 160

2. Replace the paranteses with the result
1+1+160

3. Calculate the result
1+1+160 = 162


adimlarida modele buldurarak yaptirabiliiriz

In [32]:
prompt = f"""
Give me the steps I need to follow to calculate the following?
1+1+(2*2^2(2+3+3*5))=?
"""

response = get_completion(prompt)
print(response)

1. First, calculate the value within the innermost parentheses:
   2 * 2^2 = 2 * 4 = 8

2. Now calculate the value within the next set of parentheses:
   2 + 3 + 3 * 5 = 2 + 3 + 15 = 20

3. Substitute the calculated values back into the original equation:
   1 + 1 + (8 * 20)

4. Next, calculate the value within the outermost set of parentheses:
   8 * 20 = 160

5. Substitute the calculated value back into the original equation:
   1 + 1 + 160

6. Finally, calculate the final sum:
   1 + 1 + 160 = 162

So, the final answer to the equation 1+1+(2*2^2(2+3+3*5)) is 162.


## 5.) Hallucination

In [39]:
prompt = f"""
What is Coskun As A Service offering from adesso?
"""

response = get_completion(prompt)
print(response)

Coskun As A Service is a cloud-based platform offered by adesso that provides a range of services for businesses, including data management, analytics, and artificial intelligence. It allows companies to access and utilize advanced technologies without the need for extensive in-house resources. This platform aims to help businesses improve their operations, decision-making, and customer experiences through the use of cutting-edge technology.
